## Imports

In [1]:
import math
from src.model_evaluation import load_model, evaluate_model, generate_text
from src.model_training import RNNLanguageModel, LSTMLanguageModel
import numpy as np
import os
import pickle
import random
import torch
import torch.nn as nn

## Setup

In [2]:
# Device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False
random.seed(seed)
np.random.seed(seed)

## Load Data

In [3]:
with open(os.path.normpath(os.path.join("data", "word_tokenisation_reuters_data.pkl")), "rb") as f:
    word_tokenised_numericalised_docs = pickle.load(f)

with open(os.path.normpath(os.path.join("data", "word_tokenisation_reuters_train_vocab.pkl")), "rb") as f:
    word_tokenisation_train_vocab = pickle.load(f)

with open(os.path.normpath(os.path.join("data", "subword_tokenisation_reuters_data.pkl")), "rb") as f:
    subword_tokenised_numericalised_docs = pickle.load(f)

with open(os.path.normpath(os.path.join("data", "subword_tokenisation_reuters_train_vocab.pkl")), "rb") as f:
    subword_tokenisation_train_vocab = pickle.load(f)

## Set Hyperparameters

In [4]:
EMBEDDING_SIZE = 128
HIDDEN_SIZE = 256
NUM_LAYERS = 2
DROPOUT = 0.0
BATCH_SIZE = 32

## RNN

### Word Tokenisation, Context Length = 16

In [5]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_word_tokenised_context_16_filename = "model_weights_rnn_word_tokens_True_context_16_20250903-112215"

rnn_word_tokenised_context_16 = load_model(model_initialised=rnn, filename=rnn_word_tokenised_context_16_filename, device=device)

In [6]:
# Cross Entropy Loss
rnn_word_tokenised_context_16_cross_entropy_loss = evaluate_model(
    model=rnn_word_tokenised_context_16,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=16,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_word_tokenised_context_16_cross_entropy_loss}")

# Perplexity
rnn_word_tokenised_context_16_perplexity = math.exp(rnn_word_tokenised_context_16_cross_entropy_loss)
print(f"Perplexity: {rnn_word_tokenised_context_16_perplexity}")

100%|██████████| 1194/1194 [00:25<00:00, 46.41it/s]

Cross Entropy Loss: 8.835175214700364
Perplexity: 6871.757773390407


In [7]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_word_tokenised_context_16_generated_sequence = generate_text(
        model=rnn_word_tokenised_context_16,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_word_tokenised_context_16_generated_sequence)
    print()

100%|██████████| 300/300 [00:00<00:00, 508.69it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today . stoltenberg said that , partly due to corporate restructuring , the official app news agency said . the industry analysts said that refiners may have been cut but the purchase price was financed . 88 pct from 16 . 3 mln barrels to acquire ( 25 ) 120-150 . 9 mln vs 481 . 5 mln avg shrs 4 , 565 , 752 vs 4 , 883 , 711 note 1986 nine month , datacard was concerned about the u . s . , yaacobi said he expects israel to extend its recent trend toward lower current market conditions . joske ' s has 26 stores in texas and other issues build as reagan ' s federal systems division , which has been hit with several executive defections , may also be put on the block . contel officials declined to more up to 30 pct on the same period in 1985 . the company ' s life insurance result fell to 365 . 7 mln guilders after 428 . 4 mln in 1985 . at lme , and use of documents will be the trading issue is likely to face how what he will raise its

100%|██████████| 300/300 [00:00<00:00, 552.00it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today for these business and unfavorable lease comitments . 1987 oper net 169 . 6 mln vs 36 . 4 mln sales 12 . 5 mln vs 4 . 6 mln speciality chemicals 6 . 2 mln vs 54 . 0 mln 12 mths shr 1 . 81 dlrs vs 1 . 18 dlrs net 20 . 3 mln vs 439 , 000 note company went public in the light of the precipitous falls on october 14 . 50 yen in tokyo - brokers phillips , has made little difference whether the tariffs were growing at shipment , and despite the provincial money making its own common shares . on capital prices for a year-on-year more increase in its restructuring . the company said results will acquire all of cf industries inc said usda of the electronics action was unchanged , on expansion of money market rates . we should pay particular attention to the acquisition would lift jannock imaging revenues against the agreement on louvre help in the continuing market , where while a delegation for penetrating markets to only contact see

100%|██████████| 300/300 [00:00<00:00, 520.06it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today for any move seen only to shareholders , he said . the terms of the last meeting was as five days of their bilateral crop in washington after december 1 , 1987 . and producer of telephone answering machines . for 1986 . j . terence murray , chairman and canada , along last year . we are street they expect attacks on goods by both canada and it is unlikely that the parts catalog division wholesales parts and needles to june and feed , aid japanese oil at trade above 1 . 87 production rates fell to 200 . 6 pct this week . the cut might imposed on many if after washington ' s going to be more rain and efficiency and on the bidding . trade deficit and they is in direct contravention of the government ' s most vexing and persistent problems . the six months in a surplus at 3 . 15 mln in 1985 , while gas demand and has five to 51 pct expansion of british printing and communication inc> 1st qtr net shr 75 cts vs 64 cts net 11 . 9 m

### Word Tokenisation, Context Length = 32

In [9]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_word_tokenised_context_32_filename = "model_weights_rnn_word_tokens_True_context_32_20250903-090142"

rnn_word_tokenised_context_32 = load_model(model_initialised=rnn, filename=rnn_word_tokenised_context_32_filename, device=device)

In [10]:
# Cross Entropy Loss
rnn_word_tokenised_context_32_cross_entropy_loss = evaluate_model(
    model=rnn_word_tokenised_context_32,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=32,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_word_tokenised_context_32_cross_entropy_loss}")

# Perplexity
rnn_word_tokenised_context_32_perplexity = math.exp(rnn_word_tokenised_context_32_cross_entropy_loss)
print(f"Perplexity: {rnn_word_tokenised_context_32_perplexity}")

100%|██████████| 1073/1073 [00:41<00:00, 26.11it/s]

Cross Entropy Loss: 10.025398466093787
Perplexity: 22593.069209792488


In [11]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_word_tokenised_context_32_generated_sequence = generate_text(
        model=rnn_word_tokenised_context_32,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_word_tokenised_context_32_generated_sequence)
    print()

100%|██████████| 300/300 [00:00<00:00, 506.68it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today can ' t be a currency , they said . the tender calls for shipment of 6 , 000 tonnes . shippers predicted it will only disappear if people lose confidence in the direction with the increased loan . the sources said the ec nations were generally weak throughout the central bank council gives its opinion on this theme only every 14 days , this is hardly probable . euromark rates scarcely responded to the central bankers comments , with six months funds ending unaltered at around 3-7/8 pct . eurodollars for the same maturity rose a fractional 1/16 pct from yesterday to around 6-11/16 pct . sterling dipped to end at 2 . 952/956 marks after 2 . 960/965 last night . the swiss franc firmed slightly to 120 . 30/45 marks per 100 from 120 . 25/40 , with the yen soaring however to 1 . 2570/90 marks per 100 from 1 . 2515/35 yesterday . ems currencies saw the french franc end unchanged from last night . in the second quarter , the company

 83%|████████▎ | 248/300 [00:00<00:00, 569.27it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today , said it has acquired the investors to exchange closing , said an annual report added . it said the venture would be his proposal to acquire the 18 dlrs for the first quarter of 1987 from 2 . 78 billion a year ago after tax proceeds from several years . a further appreciation of the dollar would hinder washington ' s efforts to reduce trade surplus with the united states , as well as total imports . the transaction would be operating , by drawing members of the international cocoa organization contract have been offered four products in last week ' s election , or 62 . 2 billion pesos in 1986 . noranda had prime the etl would be lost to common stock , was the largest u . s . subsidiary , chene investments inc , bought 2 . 55 mln units . oil to 9 , 1986 , and soft currency ) but in paris on the imf ' s policy-making quota system deadline for a u . s . tenderloin steak in which u . s . meat producers claim will cut off their 

 48%|████▊     | 145/300 [00:00<00:00, 498.99it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today he is seeking low since rain ' s system rose an unadjusted to end-march to further climbs . the u . s . deficit , despite progress seen stagnating power and exaggerated problems where dealers is one reason for cautious -- v overseas sales are expected to announce it over the last 17 years , and that short-term be imposed from production of a tax held by british subsidies . speaking at this week ' s session would be convened to adopt the resolution after the first u . s . taiwan ' s gdp hit some of the burden of trade friction and economic restructuring away from the employees , the industrial country held to one of the company . zinn and certain affiliates make changes for gillette bank &lt smaller for coffee sugar by volcker and beans . trading on nasdaq 70 .



### Subword Tokenisation, Context Length = 16

In [13]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_subword_tokenised_context_16_filename = "model_weights_rnn_word_tokens_False_context_16_20250903-120755"

rnn_subword_tokenised_context_16 = load_model(model_initialised=rnn, filename=rnn_subword_tokenised_context_16_filename, device=device)

In [14]:
# Cross Entropy Loss
rnn_subword_tokenised_context_16_cross_entropy_loss = evaluate_model(
    model=rnn_subword_tokenised_context_16,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=16,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_subword_tokenised_context_16_cross_entropy_loss}")

# Perplexity
rnn_subword_tokenised_context_16_perplexity = math.exp(rnn_subword_tokenised_context_16_cross_entropy_loss)
print(f"Perplexity: {rnn_subword_tokenised_context_16_perplexity}")

100%|██████████| 1566/1566 [00:45<00:00, 34.47it/s]


Cross Entropy Loss: 7.443448190411997
Perplexity: 1708.631762647215


In [15]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_subword_tokenised_context_16_generated_sequence = generate_text(
        model=rnn_subword_tokenised_context_16,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_subword_tokenised_context_16_generated_sequence)
    print()

100%|██████████| 300/300 [00:00<00:00, 414.05it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ▁we ▁can ▁be ▁achieved 
 ▁or ▁as ▁well ▁as ▁a ▁result ▁of ▁declining 
 ▁s ept - 3 / 4 ▁p ct ▁of ▁the ▁total , ▁from ▁7 44 , 200 ▁units , ▁or ▁6 . 2 ▁p ct ▁from ▁a ▁record ▁3 . 2 ▁p ct , 
 ▁doubling ▁the ▁nominal ▁deficit ▁to ▁1 . 33 ▁0 . 50 ▁0 . 50 
 ▁end ▁stocks ▁0 . 22 ▁0 . 29 ▁0 . 52 ▁0 . 39 ▁0 . 39 ▁0 . 39 
 ▁th ailand ▁-- ▁foreign ▁exchange ▁1986 ▁marginal ▁care ▁review 
 ▁before ▁the ▁ 568 ▁m ln ▁s tg ▁contested ▁bid ▁was ▁launched ▁last ▁s ept ember 
 ▁the ▁decline ▁in ▁the ▁1987 , ▁already ▁has ▁already 
 ▁or ▁take ▁in ▁the ▁near ▁future . ▁cam pe au . 
 ▁on ▁ap ril ▁29 ▁to ▁tender ▁for ▁cash 
 ▁per ▁share ▁from ▁previous ▁the ▁total , 
 ▁based ▁on ▁ec ▁trade ▁spokesman , ▁vice ▁president ▁hel mut 
 ▁sch les inger ▁said ▁he ▁saw ▁no ▁reason ▁to ▁lower ▁interest ▁rates 
 ▁the ▁federal ▁reserve ▁entered ▁the ▁u . s . 
 ▁benchmark ▁grade ▁before ▁the ▁four 
 ▁shareholders ▁would ▁hold ▁its ▁own ▁shares ▁and ▁a ▁review ▁

 95%|█████████▌| 286/300 [00:00<00:00, 436.70it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today ▁to ▁include ▁pork 
 ▁j apan ese ▁parts , ▁raw ▁has ▁been ▁down ▁on 
 ▁the ▁proposed ▁merger ▁of ▁via com ▁says ▁it ▁has ▁often ▁going 
 ▁to ▁be ▁closed ▁by ▁j ames ▁baker . 
 ▁officials ▁say ▁the ▁infant ▁exchange ▁has ▁made ▁as ▁the 
 ▁old ▁opening , ▁national ▁food grain ▁reserves ▁being iment 
 ▁et ▁des ▁t rav aux ▁pu bl ics > ▁( b tp ) ▁has ▁emerged 
 borg - war ner & lt ; bor > ▁which ▁computer ▁associates ' 
 ▁routes , ▁and ▁it ▁( ▁60 ▁billion ▁ dl rs , ▁or ▁29 ▁c ts ▁per ▁share ▁after ▁an ▁amount ▁of 
 ▁h olly f workers ▁co ▁& lt ; j h > ▁1 st ▁q tr ▁net 
 ▁oper ▁shr ▁primary ▁87 ▁c ts ▁vs ▁three ▁c ts 
 ▁net ▁152 , 360 ▁vs ▁94 , 141 
 ▁re vs ▁4 , 675 , 9 14 ▁vs ▁1 , 035 , 162 
 ▁forecast ▁for ▁cash ▁flow ▁by ▁0 . 50 ▁0 . 50 
 ▁ dl rs ▁cumulative ▁preferred ▁stock ▁and ▁the ▁anti - inflammatory 
 ▁markets . , ▁but ▁we ▁feel ▁this ▁has ▁yet 
 ▁would ▁have ▁broken ▁the ▁spirit ▁of ▁the ▁law ▁setting ▁up 
 ▁com sat . co

  5%|▌         | 15/300 [00:00<00:00, 334.27it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today ▁inc ▁and ▁has 
 ▁received ▁accept ances ▁of ▁which ▁last ▁government ▁may ▁it ▁will ▁show



### Subword Tokenisation, Context Length = 32

In [17]:
# Initialise model
rnn = RNNLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

rnn_subword_tokenised_context_32_filename = "model_weights_rnn_word_tokens_False_context_32_20250903-101259"

rnn_subword_tokenised_context_32 = load_model(model_initialised=rnn, filename=rnn_subword_tokenised_context_32_filename, device=device)

In [18]:
# Cross Entropy Loss
rnn_subword_tokenised_context_32_cross_entropy_loss = evaluate_model(
    model=rnn_subword_tokenised_context_32,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=32,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {rnn_subword_tokenised_context_32_cross_entropy_loss}")

# Perplexity
rnn_subword_tokenised_context_32_perplexity = math.exp(rnn_subword_tokenised_context_32_cross_entropy_loss)
print(f"Perplexity: {rnn_subword_tokenised_context_32_perplexity}")

100%|██████████| 1442/1442 [00:59<00:00, 24.06it/s]


Cross Entropy Loss: 8.542993121851499
Perplexity: 5130.678147131693


In [19]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    rnn_subword_tokenised_context_32_generated_sequence = generate_text(
        model=rnn_subword_tokenised_context_32,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(rnn_subword_tokenised_context_32_generated_sequence)
    print()

100%|██████████| 300/300 [00:00<00:00, 600.45it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ," ▁said ▁fil des . 
 ▁he ▁said ▁sq ui bb ▁would ▁continue ▁to ▁1 . 34 ▁m ln 
 ▁units ▁in ▁1986 , ▁from ▁all , 000 ▁ dl rs 
 ▁against ▁results ▁for ▁12 . 5 ▁m ln ▁shares ▁of 
 ▁4 . 9 ▁m ln ▁barrels ▁per ▁day ▁( b pd ) ▁output ▁ceiling ▁agreed ▁by ▁a ▁other ▁company , ▁entertainment ▁marketing ▁and 
 ▁both ▁the ▁market ▁situation ▁that ▁the ▁fed ▁is ▁expected ▁to ▁add . 
 ▁the ▁west ▁g erman ▁government ▁is ▁expected ▁to ▁come 
 ▁this ▁year . 
 ▁" wage ▁main ▁own ▁price , ▁economists ▁said 
 ▁the ▁business ▁will ▁be ▁announced ▁on ▁the ▁trade ▁will ▁currency ▁appreciation ▁on ▁the 
 ▁substantial ▁lowest ▁of ▁the ▁discount 
 ▁energy ▁futures ▁department ▁said . 
 ▁an ▁attorney ▁for ▁ amm een , ▁who ▁is ▁named ▁as ▁it ▁dropped ▁and ▁sell ▁that ▁payments ▁since ▁consumers . 
 ▁" the ▁( can ada ) ▁ pc ▁and ▁said ▁the ▁study ▁and ▁is ▁subject ▁to 
 ▁ceilings ▁determined ▁and ▁expand 
 ▁the ▁energy ▁information ▁spokesman ▁said ▁th

100%|██████████| 300/300 [00:00<00:00, 731.39it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today ," ▁said 
 ▁1985 , ▁z ▁stock ▁has ▁given ▁first ▁may ▁be 
 ▁bid ▁for ▁its ▁offer ▁on ▁behalf ▁of ▁an ▁improvement ▁to ▁differ entials ▁making ▁an ▁agreement 
 ▁holding ▁company ▁( for ▁d ana ▁& lt ; gr os la ▁are ▁still 
 ▁available ▁to ▁a ▁new ▁disillusion ment ▁with ▁the ▁current 
 ▁and ▁areas ▁and ▁will ▁be ▁paid , ▁for ▁market ▁nerves ▁about ▁the ▁poll ▁outcome , ▁they ▁said . 
 ▁they ▁said ▁some ▁char les ▁red ▁futures ▁is ▁the ▁largest ▁of ▁the 
 ▁state ▁statistics ▁be ▁res ched uled , ▁with ▁the ▁t ran scan ada ▁offer , ▁the ▁company ▁said ▁it ▁sold 
 ▁666 , 667 ▁shares ▁at ▁75 ▁c ts ▁a ▁share . 
 ▁m ead ▁corp ▁said 
 ▁its ▁royal ▁d utch ▁consumer ▁vessels ▁stock , ▁has ▁been ▁speculation ▁that ▁the ▁consumer ▁position ▁has ▁not 
 ▁tightened ▁credit ▁to ▁the ▁degree ▁suggested ▁by ▁the ▁data 
 ▁locals ▁would ▁meet ▁on ▁the ▁other ▁two ▁lower , 
 ▁which ▁is ▁in ▁a ▁fifth ▁year ▁of ▁v ancouver , ▁said ▁the ▁sales ▁are 


 73%|███████▎  | 219/300 [00:00<00:00, 636.27it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today : ▁consolidated ) ▁pact ▁will ▁be ▁converted ▁into ▁1 . 79 ▁f ran ▁van ▁dy ke ▁calculated 
 ▁to ▁meet ▁the ▁25 ▁billion ▁up ▁rate ▁rate ▁yesterday ▁at ▁underlying ▁price ▁for ▁2 ▁of ▁government ▁bills ▁in ▁the ▁cane ▁areas . ▁ 
 ▁the ▁company ▁said ▁the ▁governor ▁is ▁rising ▁a ▁detailed ▁plan ▁calling ▁will 
 ▁qualify , ▁although ▁according ▁to ▁a ▁deficit ▁of ▁2 . 20 ▁p ct ▁v ob . 
 ▁the ▁joint ▁committee ▁of ▁an ▁earthquake ▁is ▁just ▁last ▁year ▁in ▁year - on - year ▁total 
 ▁about ▁69 , 000 ▁long ▁sales ▁of ▁may owment 
 ▁unless ▁the ▁u . s ., ▁chairman ▁said ▁last ▁month ▁it ▁had 
 ▁helped ▁dispel ▁inflation . ▁" it ▁has ▁become ▁the ▁purchase ▁price 
 ▁and ▁one ▁60 ▁days ▁after ▁agreeing ▁a 
 ▁share . 
 ▁as ▁will ▁be cor 's ▁b ▁adjustments . 
 ▁pre tax ▁175 - 190 ▁days ▁5 . 85 ▁p ct ▁5 . 82 ▁p ct ▁88 ▁m ln ▁ dl rs ▁and 
 ▁deposits ▁grew ▁14 ▁p ct ▁to ▁ 635 . 5 ▁m ln ▁guild ers ▁from ▁70 ▁m ln 
 ▁common ▁shares ▁were ▁

## LSTM

### Word Tokenisation, Context Length = 16

In [20]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_word_tokenised_context_16_filename = "model_weights_lstm_word_tokens_True_context_16_20250903-114443"

lstm_word_tokenised_context_16 = load_model(model_initialised=lstm, filename=lstm_word_tokenised_context_16_filename, device=device)

In [21]:
# Cross Entropy Loss
lstm_word_tokenised_context_16_cross_entropy_loss = evaluate_model(
    model=lstm_word_tokenised_context_16,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=16,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_word_tokenised_context_16_cross_entropy_loss}")

# Perplexity
lstm_word_tokenised_context_16_perplexity = math.exp(lstm_word_tokenised_context_16_cross_entropy_loss)
print(f"Perplexity: {lstm_word_tokenised_context_16_perplexity}")

100%|██████████| 1194/1194 [00:37<00:00, 32.13it/s]


Cross Entropy Loss: 10.4328327805952
Perplexity: 33956.40519044669


In [22]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_word_tokenised_context_16_generated_sequence = generate_text(
        model=lstm_word_tokenised_context_16,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_word_tokenised_context_16_generated_sequence)
    print()

100%|██████████| 300/300 [00:00<00:00, 350.67it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today . it denied the government ' s daily line . the major industrial democracies will take place against a backdrop of rising congressional concern over reagan ' s plan but not protectionist , he added , but a possible sale of a turnover of 12 . 4 mln dlrs . blue arrow shares were trading 9p to 1 , 300 , 000 dlrs in quarter and 6 , 200 , 000 dlrs vs 7 , 225 , 000 dlrs in quarter and 3 , 039 , 000 dlres in half . current half net includes 2 , 051 , 000 dlr pretax gain 2 , 051 , 000 dlr acquisition of winterhalter into a profitable field , statoil said . the sale , covering a 20 year period beginning may 1 , 1989 , was previously recommended by the national energy board . contract terms were not released by analysts said its board meeting has made little headway in the area of financing , the government of japan and cut south korean markets to open the door to rice imports in the u . s . over 1985 in february that cited two weeks 

100%|██████████| 300/300 [00:00<00:00, 429.67it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today as the markets concluded that the stock market ' s collapse precluded the central bank from carrying out the widely-rumoured discount rate increase . a senior government economist suggested that both the u . s . would cut its huge foreign trade deficit more rapidly than many now believed . he said asean ' s looming trade problems would not necessarily stem from protectionist legislation now being contemplated by congress , but simply because of the adjustments the u . s . economy will sustain that strength in the longer term . weak u . s . soybean world market share has declined 35 pct in volume and 40 pct in value since 1982 , primarily as the record of soybeans and because the overall foreign oil supply impact in the form of its consumer products segment , has decided against buying charmglow . while roth-american said it is still interested in acquiring turco , atcor said it is now reviewing its options with other potenti

 72%|███████▏  | 216/300 [00:00<00:00, 395.50it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today in tokyo and that interest rates are not lower for economic policy for lead is in line with the government ' s cocoa products to those than this year ' s balance of a 47 pct rise interest in the year to march 1986 earnings group ' s western energy demand , higher import policy . now . the rise was attributed to the shortage of australian goods about one pct the source of protecting tin banks and the western soybean imports of grains and oilseeds in military oil marketing . the company said backlog stood at 7 . 520 billion dlrs , loans . it last week with the german state valued at 1 , 747 , 000 units in 1986 , partly at the entire year of the recent months , and west germany , it said . orders in spain and production at 505 , 000 tonnes , it earned 11 . 3 mln dlrs , or six cts per share , and last year on non-accrual , 300 , 000 vs 8 , 652 , 000 note 1st half oper shr 36 cts vs 35 cts oper net 1 , 619 , 000 vs 1 , 700 , 000 

### Word Tokenisation, Context Length = 32

In [23]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(word_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=word_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_word_tokenised_context_32_filename = "model_weights_lstm_word_tokens_True_context_32_20250903-094435"

lstm_word_tokenised_context_32 = load_model(model_initialised=lstm, filename=lstm_word_tokenised_context_32_filename, device=device)

In [24]:
# Cross Entropy Loss
lstm_word_tokenised_context_32_cross_entropy_loss = evaluate_model(
    model=lstm_word_tokenised_context_32,
    converted_tokenised_docs=word_tokenised_numericalised_docs,
    seq_len=32,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=word_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(word_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_word_tokenised_context_32_cross_entropy_loss}")

# Perplexity
lstm_word_tokenised_context_32_perplexity = math.exp(lstm_word_tokenised_context_32_cross_entropy_loss)
print(f"Perplexity: {lstm_word_tokenised_context_32_perplexity}")

100%|██████████| 1073/1073 [01:16<00:00, 14.02it/s]

Cross Entropy Loss: 11.714816488707498
Perplexity: 122371.46449961234


In [25]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_word_tokenised_context_32_generated_sequence = generate_text(
        model=lstm_word_tokenised_context_32,
        train_vocab=word_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_word_tokenised_context_32_generated_sequence)
    print()

100%|██████████| 300/300 [00:01<00:00, 293.95it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ' s meetings of international trade and industry ( miti ) recommended this week that the government press ahead with deregulation of the oil industry , including the lifting of gasoline production quotas , to help oil companies increase their international competitiveness . japan ' s three biggest gasoline marketers , nippon oil , idemitsu and showa shell , suffered supply shortages under the production quota guideline in fiscal 1986 , ended last march . domestic gasoline demand during the year was being after delayed this month , a little battle . as a result , o ' neill said , there will be some impact on the real economy , but it may be forced to do so in financing the deal , said sources familiar with the transaction . borg-warner has said its management is not a participant in the transaction , but managers will retain their positions . the investors involved with merrill include pension funds , insurance companies and 

100%|██████████| 300/300 [00:00<00:00, 360.42it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today to sell its celeron corp chairman and e . f . hutton said . some others , however , judged that the dollar has become more serious as a originally signal to new orleans from subsidy in coffee , officials and firm said . the opening compares with 143 . 90/144 . 00 yen average in denmark and was 1 . 13 mln tonnes , up from 1 . 97 mln tonnes , compared with an estimated offtake of 2 . 88 mln in 1985 , on 23 . 22 billion rand in april from the first four months this year from 8 . 7 billion . exports to the convertible level of tariffs on orange juice by farmers . baker we certainly really expect another one ounce this month to drop by 4 . 3 pct one year ago . theodore brophy , chairman of gte said , we expect us sprint ' s losses to diminish later this year as customer traffic migrates to the new fiber-optic network for long distance telecommunications . the reason for the higher losses reflect lower prices as well as higher ope

 99%|█████████▊| 296/300 [00:00<00:00, 401.16it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today ' s shortfall to record stocks . seasonally adjusted spending in europe and two , and one of the products under basic income , currently which previously &lt 44 . o> 3rd qtr loss shr loss 22 cts vs loss 57 cts net loss 3 , 609 , 000 vs loss 240 , 000 revs 33 , 482 vs 1 , 782 , 047 note 1987 vs 257 mln building half note share adjusted for seven mln stg vs 22 . 0 mln santos management , the officers of datron corp &lt pa> and inland steel corp &lt s> . doe said completion of it ' s revenues for sales of 34 . 7 mln dlrs . these operations were a bank of public plants . the amounts of what in the u . s . received to the u . s . last year , dominion unsuccessfully bid for avondale mills and has maintained a 120 mln dlr line of credit to be far back before the results meeting in february , the central bank council gives its weekly proposal on a new plan for shipments , the spokesman added . the deal , which has been due to change

### Subword Tokenisation, Context Length = 16

In [26]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_subword_tokenised_context_16_filename = "model_weights_lstm_word_tokens_False_context_16_20250903-123204"

lstm_subword_tokenised_context_16 = load_model(model_initialised=lstm, filename=lstm_subword_tokenised_context_16_filename, device=device)

In [27]:
# Cross Entropy Loss
lstm_subword_tokenised_context_16_cross_entropy_loss = evaluate_model(
    model=lstm_subword_tokenised_context_16,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=16,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_subword_tokenised_context_16_cross_entropy_loss}")

# Perplexity
lstm_subword_tokenised_context_16_perplexity = math.exp(lstm_subword_tokenised_context_16_cross_entropy_loss)
print(f"Perplexity: {lstm_subword_tokenised_context_16_perplexity}")

100%|██████████| 1566/1566 [00:57<00:00, 27.45it/s]

Cross Entropy Loss: 9.117478087753781
Perplexity: 9113.189946586552


In [28]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_subword_tokenised_context_16_generated_sequence = generate_text(
        model=lstm_subword_tokenised_context_16,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_subword_tokenised_context_16_generated_sequence)
    print()

100%|██████████| 300/300 [00:01<00:00, 282.05it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ," ▁said ▁the ▁united ▁states ▁was 
 ▁prepared ▁for ▁a ▁major ▁war ▁imposed ▁by ▁congress . 
 ▁if ▁he ▁thought ▁the ▁chances ▁of ▁it ▁would ▁consider ▁leaving ▁the ▁ ico 
 ▁if ▁we ▁would ▁not ▁only ▁want ▁to ▁open ▁the ▁way ▁for ▁another ▁non - v oting ▁basis . 
 ▁based ▁in ▁brist ol , ▁eng land , ▁i sis ▁employs ▁65 ▁workers ▁and ▁has 
 ▁annual ▁revenues ▁of ▁1 . 9 ▁m ln ▁ dl rs ▁in ▁year ▁and ▁loss ▁of ▁9 . 2 ▁m ln ▁ dl rs , ▁or ▁54 ▁c ts ▁per ▁share , ▁in 
 ▁1986 . 
 ▁the ▁bank ▁of ▁eng land ▁said ▁it ▁forecast ▁a 
 ▁shortage ▁of ▁around ▁250 ▁m ln ▁s tg ▁in ▁external 
 ▁ bor ro wings . 
 ▁lex ▁shares ▁fell ▁by ▁75 ▁m ln ▁s tg ▁help ▁in ▁the ▁morning ▁session . 
 ▁this ▁compares ▁with ▁the ▁bank 's ▁estimate ▁that ▁the ▁system ▁would 
 ▁face ▁a ▁shortage ▁of ▁around ▁400 ▁m ln ▁s tg ▁in ▁the ▁money ▁market ▁today . 
 ▁among ▁the ▁main ▁factors ▁affecting ▁liquidity , ▁bills ▁mat uring 
 ▁in ▁official ▁hands ▁and ▁the ▁tak

 27%|██▋       | 81/300 [00:00<00:00, 358.77it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today ," ▁said ▁the ▁re agan ▁administration 
 ▁council ▁with ▁rumors ▁its ▁commitments ▁will ▁be ▁seeking 
 ▁in ▁late ▁aug ust . 
 ▁a ▁d utch ▁cash ▁subsidiary ▁said ▁it ▁expects ▁to ▁make ▁the ▁payment ▁due ▁due ▁to 
 ▁weakness ▁for ▁the ▁first ▁six ▁years ▁at ▁105 ▁p ct , ▁during ▁the ▁seventh ▁year ▁at 
 ▁102 . 60 / 5 ▁at ▁a ▁month ▁earlier . ▁the ▁bank ▁of ▁e tl ▁and ▁its 
 ▁consolidated ▁primary ▁60 - h wan ▁1 ▁gas



100%|██████████| 300/300 [00:00<00:00, 397.60it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today ." 
 ▁official ▁figures ▁show ▁the ▁balance , ▁products ▁in ▁each 
 ▁in ▁the ▁ phi lip pine ▁business . ▁ 
 ▁j us tin ▁si lv erton , ▁equity ▁economist ▁at ▁credit ▁su isse ▁said . 
 ▁the ▁partners ▁were ▁at ▁about ▁three ▁times 
 ▁in ▁the ▁n ether lands ' ▁spokesman ▁ger ▁committee , ▁economic ▁growth ▁and ▁ind 
 ▁re apan ▁said . 
 ▁m iller ▁said ▁shareholders ▁will ▁be ▁followed ▁by ▁a ▁merger ▁in ▁which ▁ asher . 
 ▁c pc ▁said ▁it ▁would ▁retain ▁automatic ▁toll ▁assets 
 ▁worth ▁about ▁nine ▁m ln ▁ dl rs . 
 ▁assets ▁of ▁the ▁shares ▁trading ▁would ▁run ▁before ▁he ▁he ▁sees ▁federal ▁savings ▁basis ▁to ▁take ▁t tl ▁share ▁at ▁the ▁end ▁of ▁24 . 
 ▁the ▁u . s . ▁imposed ▁100 ▁p ct ▁import ▁duties ▁on ▁personal 
 ▁computers , ▁colour ▁telev isions ▁and ▁power ▁tools , ▁alleging ▁that 
 ▁j apan ▁had ▁violated ▁last ▁s aturday ▁might ▁be ▁sold , 
 ▁m unch meyer , ▁ heng - 74 ▁long ▁term ▁sugar . 
 ▁the ▁country 's ▁foreign 

### Subword Tokenisation, Context Length = 32

In [29]:
# Initialise model
lstm = LSTMLanguageModel(
    vocab_size=len(subword_tokenisation_train_vocab),
    embed_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pad_idx=subword_tokenisation_train_vocab["<pad>"]
).to(device)

lstm_subword_tokenised_context_32_filename = "model_weights_lstm_word_tokens_False_context_32_20250903-104701"

lstm_subword_tokenised_context_32 = load_model(model_initialised=lstm, filename=lstm_subword_tokenised_context_32_filename, device=device)

In [30]:
# Cross Entropy Loss
lstm_subword_tokenised_context_32_cross_entropy_loss = evaluate_model(
    model=lstm_subword_tokenised_context_32,
    converted_tokenised_docs=subword_tokenised_numericalised_docs,
    seq_len=32,
    batch_size=BATCH_SIZE,
    criterion=nn.CrossEntropyLoss(ignore_index=subword_tokenisation_train_vocab["<pad>"]),
    device=device,
    vocab_size=len(subword_tokenisation_train_vocab)
)
print(f"Cross Entropy Loss: {lstm_subword_tokenised_context_32_cross_entropy_loss}")

# Perplexity
lstm_subword_tokenised_context_32_perplexity = math.exp(lstm_subword_tokenised_context_32_cross_entropy_loss)
print(f"Perplexity: {lstm_subword_tokenised_context_32_perplexity}")

100%|██████████| 1442/1442 [01:39<00:00, 14.53it/s]

Cross Entropy Loss: 10.090113407943848
Perplexity: 24103.525809895324


In [31]:
# Generate text
starting_sequence = ["Today"]
temperatures = [0.7, 1.0, 1.3]

for temperature in temperatures:
    lstm_subword_tokenised_context_32_generated_sequence = generate_text(
        model=lstm_subword_tokenised_context_32,
        train_vocab=subword_tokenisation_train_vocab,
        start_seq=starting_sequence,
        temperature=temperature
    )
    print(f"Generated Sequence from {starting_sequence} with temperature {temperature}:")
    print(lstm_subword_tokenised_context_32_generated_sequence)
    print()

 59%|█████▉    | 177/300 [00:00<00:00, 269.30it/s]


Generated Sequence from ['Today'] with temperature 0.7:
Today ▁to ▁the ▁test . ▁it ▁is 
 ▁for ▁bur lington ▁i 's ▁there ▁was ▁that ▁questions ▁that ▁it 
 ▁will ▁issue ▁the ▁tariffs ▁on ▁j apan ese ▁investors ▁from ▁rapid ▁two 
 ▁other ▁other ▁countries ▁like ▁beef ▁imports . 
 ▁" that ▁will ▁also ▁become ▁the ▁largest ▁single ▁action ▁is ▁not 
 ▁to ▁end ▁the ▁economy ▁and ▁risen ▁to ▁further ▁the ▁dollar ▁from ▁just 
 ▁further ▁8 . x ▁point ▁cut 

 ▁fed ▁says ▁decision ▁would ▁be ▁above ▁500 ▁yen , ▁the ▁floor ▁that 
 ▁it ▁would ▁also ▁increase ▁the ▁gross ▁national ▁products ▁of ▁both 
 ▁countries . 
 ▁ nis ka nen ▁said ▁the ▁goal ▁of ▁a ▁pact ▁should ▁be ▁to ▁end ▁all 
 ▁tariffs ▁within ▁10 ▁years , ▁lower ▁subsidies ▁on ▁exports , ▁set ▁rules 
 ▁for ▁trade ▁in ▁services ▁and ▁investments , ▁he ▁said . 
 ▁" the ▁lesson ▁of ▁history ▁is ▁that ▁once ▁the ▁inflation ▁ge nie ▁gets 
 ▁out ▁of ▁the ▁bottle , ▁it ▁continues ▁to ▁persist ," ▁he ▁said , ▁adding ▁he 
 ▁would ▁like ▁the ▁fed ▁t

100%|██████████| 300/300 [00:00<00:00, 379.32it/s]


Generated Sequence from ['Today'] with temperature 1.0:
Today ▁for ▁j apan ▁from ▁getting ▁an ▁lawsuits 's ▁proposal . 
 ▁in ▁its ▁statement , ▁ ku wait ▁oil ▁investors ▁has ▁sold ▁1 , 500 ▁and ▁to ▁1 , 500 
 ▁beds , ▁in ▁bel g ium , ▁ eu rop ean ▁country 

 ▁us ▁pay ▁aug ust ▁to ▁meet ▁south ▁k orean 
 ▁prime ▁company . 
 ▁the ▁reform ▁of ▁three ▁leading ▁ ku wait ▁was ▁not ▁immediately 
 ▁avail al be ▁must ▁be ▁around ▁cost ▁to 
 ▁commission 's ▁markets ▁that ▁was ▁trying ▁to ▁sell ▁its ▁u . s . ▁acquisition . 
 ▁the ▁company ▁said ▁the ▁action ▁affects ▁401 ▁m ln 
 ▁ dl rs ▁of ▁loans ▁to ▁b razil ▁and ▁73 ▁m ln ▁of ▁loans ▁to ▁ec uador . 
 ▁b razil ▁suspended ▁interest ▁payments ▁on ▁its ▁68 ▁billion ▁ dl rs 
 ▁of ▁medium - ▁and ▁long - term ▁debt ▁on ▁f eb ruary ▁22 . 
 ▁u . s . ▁banking ▁data ▁suggest ▁the ▁industry 
 ▁including ▁f eb ruary ▁1986 . ▁this ▁is ▁quoted ▁the ▁government ▁to ▁remove 
 ▁residual ▁import ▁quota ▁items , ▁and ▁markets ▁are ▁considered ▁by ▁ic mc 's ▁nil 


 17%|█▋        | 50/300 [00:00<00:00, 402.81it/s]

Generated Sequence from ['Today'] with temperature 1.3:
Today ▁likely ▁to ▁follow ▁the ▁war ▁agree ▁to ▁the 
 ▁market ▁in ▁return ▁to ▁the ▁new 
 ▁industry ," ▁he ▁said . 
 ▁he ▁also ▁told ▁reporters ▁that ▁many ▁major ▁nations ▁yesterday 
 ▁intervened ▁heavily ▁to ▁support ▁the ▁dollar ▁against ▁the ▁yen ▁or ▁came ," ▁said 
 ▁n eda ▁said .

